# TensorLogic Python Tutorial: Advanced Topics

This notebook covers advanced features of TensorLogic for experienced users.

## Topics Covered

1. Multi-arity predicates and relational reasoning
2. Nested quantifiers and complex queries
3. Performance optimization and graph inspection
4. Strategy selection for different use cases
5. Integration patterns
6. Error handling and debugging
7. Best practices and tips

## Setup

In [ ]:
import pytensorlogic as tl
import numpy as np
import matplotlib.pyplot as plt
import time
from typing import Dict, List, Tuple

print(f"TensorLogic version: {tl.__version__}")

## 1. Multi-Arity Predicates

### 1.1 Binary Predicates

Binary predicates represent relations between two entities:

In [ ]:
# Define binary predicate: knows(x, y)
x = tl.var("x")
y = tl.var("y")
knows = tl.pred("knows", [x, y])

# Compile and execute
graph = tl.compile(knows)

# Knowledge graph: 3x3 matrix (who knows whom)
# 1.0 = definitely knows, 0.0 = doesn't know, 0.5 = might know
knows_data = np.array([
    [0.0, 1.0, 0.5],  # Person 0 knows person 1, might know person 2
    [1.0, 0.0, 1.0],  # Person 1 knows persons 0 and 2
    [0.5, 1.0, 0.0],  # Person 2 knows person 1, might know person 0
])

result = tl.execute(graph, {"knows": knows_data})

print("Knows relation matrix:")
print(knows_data)
print(f"\nGraph execution result shape: {result.shape}")

### 1.2 Ternary Predicates

Ternary predicates involve three entities:

In [ ]:
# Define ternary predicate: gave(giver, item, receiver)
giver = tl.var("giver")
item = tl.var("item")
receiver = tl.var("receiver")
gave = tl.pred("gave", [giver, item, receiver])

# Compile
graph = tl.compile(gave)

# 3D tensor: 2 people × 3 items × 2 people
# gave[i,j,k] = probability that person i gave item j to person k
gave_data = np.random.rand(2, 3, 2)

result = tl.execute(graph, {"gave": gave_data})

print(f"Gave relation tensor shape: {gave_data.shape}")
print(f"Result shape: {result.shape}")
print(f"\nExample: Person 0 gave item 1 to person 1 with probability {gave_data[0,1,1]:.3f}")

### 1.3 Complex Query: Transitive Closure

Query: If A knows B and B knows C, does A indirectly know C?

In [ ]:
# Define variables
a = tl.var("a")
b = tl.var("b")
c = tl.var("c")

# Predicates
knows_ab = tl.pred("knows_ab", [a, b])
knows_bc = tl.pred("knows_bc", [b, c])

# Query: EXISTS b. (knows(a,b) AND knows(b,c))
query = tl.exists("b", "Person", tl.and_expr(knows_ab, knows_bc))

# Compile
graph = tl.compile(query)

# Knowledge matrix (3 people)
knows_matrix = np.array([
    [0.0, 0.9, 0.0],  # Person 0 knows person 1
    [0.0, 0.0, 0.8],  # Person 1 knows person 2
    [0.0, 0.0, 0.0],  # Person 2 knows nobody
])

# Execute: for each (a,c) pair, check if there's an intermediate b
result = tl.execute(graph, {"knows_ab": knows_matrix, "knows_bc": knows_matrix})

print("Direct knowledge:")
print(knows_matrix)
print(f"\nIndirect knowledge (via one intermediary):")
print(result.reshape(3, 3))
print(f"\nPerson 0 indirectly knows person 2 with probability {result.reshape(3,3)[0,2]:.3f}")
print(f"(via person 1: 0.9 * 0.8 = {0.9 * 0.8:.3f})")

## 2. Nested Quantifiers

### 2.1 Double Quantification

Query: FORALL x. EXISTS y. knows(x, y)

"Everyone knows at least one person"

In [ ]:
# Define variables
x = tl.var("x")
y = tl.var("y")

# Predicate
knows = tl.pred("knows", [x, y])

# EXISTS y. knows(x, y) - "x knows at least one person"
exists_knows = tl.exists("y", "Person", knows)

# FORALL x. EXISTS y. knows(x, y) - "everyone knows at least one person"
forall_exists = tl.forall("x", "Person", exists_knows)

# Compile
graph = tl.compile(forall_exists)

# Test scenarios
scenarios = [
    ("Everyone connected", np.array([
        [0.0, 0.9, 0.8],
        [0.9, 0.0, 0.7],
        [0.8, 0.7, 0.0],
    ])),
    ("One isolated", np.array([
        [0.0, 0.9, 0.0],
        [0.9, 0.0, 0.8],
        [0.0, 0.0, 0.0],  # Person 2 is isolated
    ])),
]

for name, knows_data in scenarios:
    result = tl.execute(graph, {"knows": knows_data})
    print(f"{name}:")
    print(knows_data)
    print(f"FORALL x. EXISTS y. knows(x,y): {result[0]:.3f}")
    print()

### 2.2 Triple Quantification

Query: FORALL x. EXISTS y. FORALL z. (knows(x,y) -> knows(y,z))

"Everyone has at least one friend who knows everyone"

In [ ]:
# This is complex! Let's build it step by step

x = tl.var("x")
y = tl.var("y")
z = tl.var("z")

knows_xy = tl.pred("knows_xy", [x, y])
knows_yz = tl.pred("knows_yz", [y, z])

# knows(x,y) -> knows(y,z)
implication = tl.imply(knows_xy, knows_yz)

# FORALL z. (knows(x,y) -> knows(y,z))
forall_z = tl.forall("z", "Person", implication)

# EXISTS y. FORALL z. (knows(x,y) -> knows(y,z))
exists_y = tl.exists("y", "Person", forall_z)

# FORALL x. EXISTS y. FORALL z. (knows(x,y) -> knows(y,z))
query = tl.forall("x", "Person", exists_y)

# Compile
graph = tl.compile(query)

# Knowledge matrix with one "super-connector" (person 1)
knows_data = np.array([
    [0.0, 0.9, 0.5],  # Person 0 knows person 1
    [0.9, 0.0, 0.9],  # Person 1 knows everyone
    [0.5, 0.9, 0.0],  # Person 2 knows person 1
])

result = tl.execute(graph, {"knows_xy": knows_data, "knows_yz": knows_data})

print("Knowledge matrix:")
print(knows_data)
print(f"\nQuery result: {result[0]:.3f}")
print("Interpretation: Does everyone have a super-connector friend?")

## 3. Performance Optimization

### 3.1 Graph Inspection

Understand the compiled graph structure:

In [ ]:
# Create a complex expression
x = tl.var("x")
y = tl.var("y")
p = tl.pred("P", [x])
q = tl.pred("Q", [y])
r = tl.pred("R", [x, y])

# Complex query: (P(x) AND Q(y)) OR R(x,y)
and_pq = tl.and_expr(p, q)
expr = tl.or_expr(and_pq, r)

# Compile
graph = tl.compile(expr)

# Inspect graph statistics
stats = graph.stats()

print("Graph Statistics:")
print("="*40)
for key, value in stats.items():
    print(f"{key:20} : {value}")

print("\nInterpretation:")
print("- tensors: Number of tensor operands")
print("- operations: Number of operations in the graph")
print("- total_nodes: Total computation nodes")

### 3.2 Benchmarking Strategies

Compare compilation and execution time across strategies:

In [ ]:
def benchmark_strategy(strategy_name: str, config: tl.CompilationConfig, 
                      expr: tl.TLExpr, inputs: Dict[str, np.ndarray], 
                      n_trials: int = 100) -> Tuple[float, float]:
    """Benchmark compilation and execution time."""
    
    # Benchmark compilation
    compile_times = []
    for _ in range(n_trials):
        start = time.perf_counter()
        graph = tl.compile_with_config(expr, config)
        compile_times.append(time.perf_counter() - start)
    
    # Benchmark execution
    graph = tl.compile_with_config(expr, config)
    exec_times = []
    for _ in range(n_trials):
        start = time.perf_counter()
        result = tl.execute(graph, inputs)
        exec_times.append(time.perf_counter() - start)
    
    return np.mean(compile_times), np.mean(exec_times)

# Create test expression
x = tl.var("x")
p = tl.pred("P", [x])
q = tl.pred("Q", [x])
r = tl.pred("R", [x])
expr = tl.and_expr(tl.or_expr(p, q), tl.not_expr(r))

# Test data (100 elements)
inputs = {
    "P": np.random.rand(100),
    "Q": np.random.rand(100),
    "R": np.random.rand(100),
}

# Benchmark strategies
strategies = [
    ("Soft Differentiable", tl.CompilationConfig()),
    ("Hard Boolean", tl.CompilationConfig()),
    ("Fuzzy Gödel", tl.CompilationConfig()),
]

results = {}
print("Benchmarking strategies (100 trials each):")
print("="*60)
print(f"{'Strategy':<20} {'Compile (μs)':<15} {'Execute (μs)':<15}")
print("-"*60)

for name, config in strategies:
    compile_time, exec_time = benchmark_strategy(name, config, expr, inputs, n_trials=100)
    results[name] = (compile_time, exec_time)
    print(f"{name:<20} {compile_time*1e6:<15.2f} {exec_time*1e6:<15.2f}")

### 3.3 Visualize Performance

In [ ]:
# Extract data
strategy_names = list(results.keys())
compile_times = [results[s][0] * 1e6 for s in strategy_names]  # Convert to μs
exec_times = [results[s][1] * 1e6 for s in strategy_names]

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Compilation time
ax1.bar(strategy_names, compile_times, color='blue', alpha=0.7)
ax1.set_ylabel('Time (μs)', fontsize=12)
ax1.set_title('Compilation Time', fontsize=14)
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)

# Execution time
ax2.bar(strategy_names, exec_times, color='green', alpha=0.7)
ax2.set_ylabel('Time (μs)', fontsize=12)
ax2.set_title('Execution Time', fontsize=14)
ax2.tick_params(axis='x', rotation=45)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Strategy Selection Guide

### When to Use Each Strategy

| Strategy | Use Case | Pros | Cons |
|----------|----------|------|------|
| **soft_differentiable** | Neural-symbolic AI, training | Smooth gradients, differentiable | May not satisfy crisp logic laws |
| **hard_boolean** | Classical logic, verification | Exact boolean semantics | Not differentiable, binary only |
| **fuzzy_godel** | Fuzzy logic, multi-valued reasoning | Satisfies many logical laws | Different semantics than classical |
| **fuzzy_product** | Probabilistic reasoning (independence) | Intuitive product semantics | Different from classical logic |
| **fuzzy_lukasiewicz** | Resource-aware reasoning | Bounded sum/difference | Complex semantics |
| **probabilistic** | Probability theory | Probabilistic interpretation | May violate logical laws |

### Strategy Selection Examples

In [ ]:
# Example 1: Training neural-symbolic model -> soft_differentiable
print("Use Case 1: Training Neural-Symbolic Model")
print("Recommendation: soft_differentiable")
print("Reason: Smooth gradients enable backpropagation\n")

# Example 2: Verifying safety properties -> hard_boolean
print("Use Case 2: Verifying Safety Properties")
print("Recommendation: hard_boolean")
print("Reason: Exact boolean semantics ensure correctness\n")

# Example 3: Multi-valued expert system -> fuzzy_godel
print("Use Case 3: Multi-Valued Expert System")
print("Recommendation: fuzzy_godel")
print("Reason: Natural interpretation for degrees of truth\n")

# Example 4: Probabilistic knowledge graph -> probabilistic
print("Use Case 4: Probabilistic Knowledge Graph")
print("Recommendation: probabilistic")
print("Reason: Direct probabilistic interpretation\n")

## 5. Integration Patterns

### 5.1 Iterative Reasoning

Apply rules iteratively until convergence:

In [ ]:
def iterative_reasoning(graph: tl.EinsumGraph, 
                       initial_state: Dict[str, np.ndarray],
                       max_iterations: int = 10,
                       tolerance: float = 1e-6) -> np.ndarray:
    """Apply a rule iteratively until convergence."""
    
    state = initial_state.copy()
    
    for iteration in range(max_iterations):
        # Execute rule
        new_state = tl.execute(graph, state)
        
        # Check convergence
        if np.allclose(new_state, list(state.values())[0], atol=tolerance):
            print(f"Converged after {iteration+1} iterations")
            return new_state
        
        # Update state
        state = {list(state.keys())[0]: new_state}
    
    print(f"Did not converge after {max_iterations} iterations")
    return new_state

# Example: Happiness propagation
# Rule: If you have happy friends, you become happier
x = tl.var("x")
y = tl.var("y")
happy = tl.pred("happy", [y])
friends = tl.pred("friends", [x, y])
has_happy_friend = tl.exists("y", "Person", tl.and_expr(happy, friends))

graph = tl.compile(has_happy_friend)

# Initial state: one person is happy
initial_happy = np.array([1.0, 0.0, 0.0, 0.0])
friends_matrix = np.array([
    [0.0, 0.8, 0.0, 0.0],
    [0.8, 0.0, 0.9, 0.0],
    [0.0, 0.9, 0.0, 0.7],
    [0.0, 0.0, 0.7, 0.0],
])

# Run iterative reasoning
final_state = iterative_reasoning(
    graph, 
    {"happy": initial_happy, "friends": friends_matrix},
    max_iterations=10
)

print(f"\nInitial happiness: {initial_happy}")
print(f"Final happiness:   {final_state}")

### 5.2 Combining Multiple Rules

Apply multiple rules in sequence:

In [ ]:
def apply_rules(rules: List[Tuple[str, tl.EinsumGraph]],
               initial_state: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
    """Apply multiple rules in sequence."""
    
    state = initial_state.copy()
    
    for rule_name, graph in rules:
        result = tl.execute(graph, state)
        print(f"Applied rule '{rule_name}': {result}")
        # Update state with new derived facts
        state[rule_name + "_result"] = result
    
    return state

# Example: Multi-step reasoning
x = tl.var("x")
p = tl.pred("P", [x])
q = tl.pred("Q", [x])

# Rule 1: P(x) AND Q(x) -> R(x)
rule1 = tl.and_expr(p, q)
graph1 = tl.compile(rule1)

# Rule 2: NOT R(x) -> S(x)
r = tl.pred("R", [x])
rule2 = tl.not_expr(r)
graph2 = tl.compile(rule2)

# Initial state
initial_state = {
    "P": np.array([1.0, 0.8, 0.5, 0.0]),
    "Q": np.array([1.0, 0.6, 0.0, 0.0]),
}

# Apply rules
rules = [("R_derivation", graph1), ("S_derivation", graph2)]
final_state = apply_rules(rules, initial_state)

print("\nFinal state:")
for key, value in final_state.items():
    print(f"{key:20} : {value}")

## 6. Error Handling and Debugging

### 6.1 Common Errors

In [ ]:
# Error 1: Mismatched tensor shapes
try:
    x = tl.var("x")
    p = tl.pred("P", [x])
    graph = tl.compile(p)
    
    # Wrong: P expects 1D array, given 2D
    result = tl.execute(graph, {"P": np.array([[1.0, 2.0]])})
except Exception as e:
    print(f"Error 1 - Shape mismatch: {type(e).__name__}")
    print(f"Message: {str(e)}\n")

# Error 2: Missing input
try:
    x = tl.var("x")
    p = tl.pred("P", [x])
    q = tl.pred("Q", [x])
    expr = tl.and_expr(p, q)
    graph = tl.compile(expr)
    
    # Wrong: Q is missing
    result = tl.execute(graph, {"P": np.array([1.0, 2.0])})
except Exception as e:
    print(f"Error 2 - Missing input: {type(e).__name__}")
    print(f"Message: {str(e)}\n")

# Error 3: Invalid variable name
try:
    # Wrong: Empty variable name
    x = tl.var("")
except Exception as e:
    print(f"Error 3 - Invalid variable: {type(e).__name__}")
    print(f"Message: {str(e)}\n")

### 6.2 Debugging Tips

In [ ]:
def debug_expression(expr: tl.TLExpr, inputs: Dict[str, np.ndarray]) -> None:
    """Debug helper for expressions."""
    
    print("Debug Information:")
    print("="*50)
    
    # 1. Compile
    try:
        graph = tl.compile(expr)
        print("✓ Compilation successful")
    except Exception as e:
        print(f"✗ Compilation failed: {e}")
        return
    
    # 2. Inspect graph
    stats = graph.stats()
    print(f"\nGraph statistics:")
    for key, value in stats.items():
        print(f"  {key}: {value}")
    
    # 3. Check inputs
    print(f"\nProvided inputs:")
    for key, value in inputs.items():
        print(f"  {key}: shape={value.shape}, dtype={value.dtype}")
    
    # 4. Execute
    try:
        result = tl.execute(graph, inputs)
        print(f"\n✓ Execution successful")
        print(f"  Result shape: {result.shape}")
        print(f"  Result dtype: {result.dtype}")
        print(f"  Result range: [{np.min(result):.3f}, {np.max(result):.3f}]")
    except Exception as e:
        print(f"\n✗ Execution failed: {e}")

# Example usage
x = tl.var("x")
p = tl.pred("P", [x])
q = tl.pred("Q", [x])
expr = tl.and_expr(p, q)

inputs = {
    "P": np.array([0.8, 0.6, 0.4]),
    "Q": np.array([0.9, 0.5, 0.1]),
}

debug_expression(expr, inputs)

## 7. Best Practices and Tips

### 7.1 Expression Design

**DO:**
- Build expressions incrementally and test at each step
- Use meaningful variable and predicate names
- Check graph statistics for complex expressions
- Normalize input data to [0, 1] range for fuzzy/soft strategies

**DON'T:**
- Create deeply nested expressions without testing
- Mix different semantic interpretations (e.g., probabilities and truth values)
- Ignore compilation strategy implications
- Use hard_boolean with continuous values

### 7.2 Performance Tips

In [ ]:
# Tip 1: Reuse compiled graphs
x = tl.var("x")
p = tl.pred("P", [x])
q = tl.pred("Q", [x])
expr = tl.and_expr(p, q)

# GOOD: Compile once, execute many times
graph = tl.compile(expr)
for i in range(100):
    inputs = {
        "P": np.random.rand(10),
        "Q": np.random.rand(10),
    }
    result = tl.execute(graph, inputs)

print("✓ Reused compiled graph 100 times")

# Tip 2: Batch processing
# Instead of processing individual elements, batch them
batch_size = 100
p_batch = np.random.rand(batch_size)
q_batch = np.random.rand(batch_size)
result = tl.execute(graph, {"P": p_batch, "Q": q_batch})

print(f"✓ Processed batch of {batch_size} elements efficiently")

# Tip 3: Choose strategy based on requirements
print("\n✓ Strategy selection guide:")
print("  - Training? Use soft_differentiable")
print("  - Verification? Use hard_boolean")
print("  - Fuzzy logic? Use fuzzy_godel or fuzzy_product")

### 7.3 Type Safety

In [ ]:
# Using type hints for better code clarity
from typing import Dict
import numpy.typing as npt

def safe_execute(graph: tl.EinsumGraph, 
                inputs: Dict[str, npt.NDArray[np.float64]]) -> npt.NDArray[np.float64]:
    """Type-safe execution wrapper."""
    
    # Validate inputs
    for key, value in inputs.items():
        if not isinstance(value, np.ndarray):
            raise TypeError(f"Input '{key}' must be numpy array, got {type(value)}")
        if value.dtype != np.float64:
            # Convert to float64
            inputs[key] = value.astype(np.float64)
    
    # Execute
    return tl.execute(graph, inputs)

# Example usage
x = tl.var("x")
p = tl.pred("P", [x])
graph = tl.compile(p)

# This will work
result = safe_execute(graph, {"P": np.array([1.0, 2.0, 3.0])})
print(f"✓ Safe execution successful: {result}")

# This will auto-convert to float64
result = safe_execute(graph, {"P": np.array([1, 2, 3], dtype=np.int32)})
print(f"✓ Auto-converted int32 to float64: {result}")

## 8. Summary

### Key Takeaways

✅ **Multi-arity predicates** enable complex relational reasoning  
✅ **Nested quantifiers** allow sophisticated queries  
✅ **Graph inspection** helps understand compilation results  
✅ **Benchmarking** informs strategy selection  
✅ **Integration patterns** enable iterative and multi-rule reasoning  
✅ **Error handling** improves robustness  
✅ **Best practices** ensure efficient and correct usage  

### Next Steps

1. Explore real-world datasets and knowledge graphs
2. Integrate with neural network training pipelines
3. Experiment with custom compilation strategies
4. Build domain-specific reasoning systems
5. Contribute to the TensorLogic ecosystem

### Resources

- **Documentation**: README.md, CLAUDE.md
- **Tests**: `tests/` directory for usage examples
- **Examples**: `examples/` directory
- **Community**: GitHub issues and discussions